# 객관식 문제 변형(251111)

In [4]:
import os, json
from tools.pipeline.config import ONEDRIVE_PATH

with open('multiple_classify.json', 'r', encoding='utf-8') as f:
    multiple = json.load(f)

## 옳은 것

### 옳은 것  -> 옳지 않은 것 문제 중 답변 몇 개 변환할 지 정하기
- 4지선다/5지선다 나눠서 진행

In [5]:
pick_right = [p for p in multiple if p['pick'] == 'right']
# print(len(pick_wrong))
options_4 = []
options_5 = []
option_others = []

for q in pick_right:
    if len(q['options']) == 4:
        options_4.append(q)
    elif len(q['options']) == 5:
        options_5.append(q)
    else:
        option_others.append(q)

ans_num_4 = {
    2: len(options_4) // 3,
    3: len(options_4) // 3,
    4: len(options_4) // 3
}
ans_num_5 = {
    2: len(options_5) // 4,
    3: len(options_5) // 4,
    4: len(options_5) // 4,
    5: len(options_5) // 4
}
if len(options_4) % 3 != 0:
    for i in range(len(options_4) % 3):
        ans_num_4[3] += 1
if len(options_5) % 4 != 0:
    for i in range(len(options_5) % 4):
        ans_num_5[4] += 1



print("4지선다 문제 수:", len(options_4))
print(ans_num_4)
print("5지선다 문제 수:", len(options_5))
print(ans_num_5)
print("총 문제수 일치 여부:", len(options_4) + len(options_5) == len(pick_right))

4지선다 문제 수: 1228
{2: 409, 3: 410, 4: 409}
5지선다 문제 수: 1029
{2: 257, 3: 257, 4: 258, 5: 257}
총 문제수 일치 여부: True


In [6]:
option_others

[]

In [7]:
import random
# random.seed(42)

# 4지선다와 5지선다를 독립적으로 랜덤 샘플링
remaining_4 = options_4.copy()
remaining_5 = options_5.copy()
pick_right_sampling = {}

for i in range(2, 6):
    sampling_result = []
    
    # 4지선다에서 샘플링 (i가 2, 3, 4일 때만)
    if i in ans_num_4:
        random.shuffle(remaining_4)
        sampled_4 = random.sample(remaining_4, ans_num_4[i])
        sampling_result.extend(sampled_4)
        remaining_4 = [x for x in remaining_4 if x not in sampled_4]
    
    # 5지선다에서 샘플링 (i가 2, 3, 4, 5일 때)
    if i in ans_num_5:
        random.shuffle(remaining_5)
        sampled_5 = random.sample(remaining_5, ans_num_5[i])
        sampling_result.extend(sampled_5)
        remaining_5 = [x for x in remaining_5 if x not in sampled_5]
    
    pick_right_sampling[i] = sampling_result
    
    with open(os.path.join(f'7_multiple_rw/pick_right/pick_right_sampling_{i}.json'), 'w', encoding='utf-8') as f:
        json.dump(pick_right_sampling[i], f, ensure_ascii=False, indent=4)

print("4지선다 남은 문제 수:", len(remaining_4))
print("5지선다 남은 문제 수:", len(remaining_5))
print("총 문제수 일치 여부:", len(options_4) + len(options_5) == len(pick_right))

4지선다 남은 문제 수: 0
5지선다 남은 문제 수: 0
총 문제수 일치 여부: True


### 변형하기

In [8]:
from tools.core.llm_query import LLMQuery
import time
import logging
from datetime import datetime

# 로깅 설정
log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f'multiple_answer.log')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

llm = LLMQuery()
output_dir = os.path.join('7_multiple_rw/pick_right')

logger.info(f"로그 파일: {log_file}")
logger.info("=" * 80)

for i in range(2, 6):
  logger.info(f"\n[정답 {i}개 그룹] 처리 시작")
  parsed_responses = []
  not_parsed_responses = []
  no_responses = []

  for idx, p in enumerate(pick_right_sampling[i], 1):
    question_id = p['file_id'] + '_' + p['tag']
    logger.info(f"\n[{i}개 그룹] {idx}/{len(pick_right_sampling[i])} - 문제 ID: {question_id} 처리 중...")
    options_ct = len(p['options'])

    if options_ct - i == 1:
      # 옳은 답이 1개여야 하는 경우
      system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳은’ 선택지)은 그대로 유지합니다.
  3) 문제 지문을 ‘옳지 않은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현은 원형을 최대한 보존합니다.
  4) 새로운 정답은 변형 후 ‘옳지 않은’ 선택지 전부입니다(총 선택지 수가 {options_ct}개라면 {i}개).

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳지 않다: 근거 ... / ③ 옳다(원래 답): 근거 ... / ⑤ 옳지 않다: 근거 ..."
  }}"""
      
    elif options_ct - i == 0:
      # 옳은 답이 0개여야 하는 경우
      system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳은’ 선택지)을 단어 1~2개 수준의 최소 변경으로 ‘옳지 않은’ 선택지로 만듭니다. (ex. 높다 -> 낮다, 한다 -> 하지 않는다)
  2) 문제 지문을 ‘옳지 않은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현 원형을 최대한 보존합니다.
  3) 새로운 정답은 변형 후 ‘옳지 않은’ 선택지 전부입니다(총 선택지 수가 {options_ct}개라면 {i}개).
  4) 해설에는 각 선택지의 옳고 그름과 간단한 근거를 명시하세요. 특히 새로 만든 정답 1개의 변경 포인트를 밝혀주세요.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳지 않다: 근거 ... / ③ 옳지 않다: 근거 ... / ⑤ 옳지 않다(원래 답): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}

  비고
  - 원문 선택지 수가 5개가 아닐 경우에도 동일 원칙을 적용합니다(총 {options_ct-i-1}개 추가 오답, 나머지 전부 정답).
  - 사실과 상충하는 임의 창작을 피하고, 제공된 해설의 논리 범위 내에서만 최소 변경을 수행하세요."""
          
    else:
      # 옳은 답이 2개 이상인 경우
      system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳은’ 선택지)은 그대로 유지합니다.
  2) 나머지 선택지 중에서 정확히 {options_ct-i-1}개를 골라, 단어 1~2개 수준의 최소 변경으로 ‘옳지 않은’ 선택지로 만듭니다. (ex. 높다 -> 낮다, 한다 -> 하지 않는다)
  3) 문제 지문을 ‘옳지 않은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현 원형을 최대한 보존합니다.
  4) 새로운 정답은 변형 후 ‘옳지 않은’ 선택지 전부입니다(총 선택지 수가 {options_ct}개라면 {i}개).
  5) 해설에는 각 선택지의 옳고 그름과 간단한 근거를 명시하세요. 특히 새로 만든 오답 1개의 변경 포인트를 밝혀주세요.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳지 않다: 근거 ... / ③ 옳다(원래 답): 근거 ... / ⑤ 옳지 않다(변형): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}

  비고
  - 원문 선택지 수가 5개가 아닐 경우에도 동일 원칙을 적용합니다(총 {options_ct-i-1}개 추가 오답, 나머지 전부 정답).
  - 사실과 상충하는 임의 창작을 피하고, 제공된 해설의 논리 범위 내에서만 최소 변경을 수행하세요."""
    
    user_prompt = f"""
========== 다음 ===========
"""
    question_id = p['file_id'] + '_' + p['tag']
    question = p['question']
    options = p['options']
    answer = p['answer']
    explanation = p['explanation']

    user_prompt += f"""문제번호: {question_id}
  문제: {question}
  선택지: {options}
  답: {answer}
  해설: {explanation}
  """
    logger.info(f"  - API 호출 중... (모델: openai/o3)")
    time.sleep(0.6)
    try:
      response = llm.query_openrouter(system_prompt, user_prompt, model_name='openai/o3')
      logger.info(f"  - API 응답 수신 완료 (길이: {len(response) if response else 0}자)")
      try:
        # 단일 JSON 객체 파싱 시도
        parsed_response = None
        # 먼저 배열 형태로 시도
        parsed_response = llm.parse_api_response(response)
        
        # 배열이 아니면 단일 객체로 시도
        if parsed_response is None:
          try:
            # JSON 객체 찾기
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx != -1 and end_idx > 0:
              json_str = response[start_idx:end_idx]
              parsed_response = json.loads(json_str)
              logger.info(f"  - 단일 JSON 객체 파싱 성공")
          except Exception as parse_err:
            logger.warning(f"  - JSON 객체 파싱 실패: {parse_err}")
            pass
        
        if parsed_response is not None:
          parsed_responses.append(parsed_response)
          logger.info(f"  ✓ 파싱 성공 - result.json에 저장")

          # 기존 파일이 있으면 읽어서 누적
          result_file = f'{output_dir}/{i}/result.json'
          os.makedirs(os.path.dirname(result_file), exist_ok=True)
          
          existing_results = []
          if os.path.exists(result_file):
            try:
              with open(result_file, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)
                # 기존 데이터가 리스트면 그대로 사용, 아니면 리스트로 변환
                if isinstance(existing_data, list):
                  existing_results = existing_data
                else:
                  existing_results = [existing_data]
            except:
              existing_results = []
          
          # 새 결과 추가
          existing_results.append(parsed_response)
          
          # 누적된 결과 저장
          with open(result_file, 'w', encoding='utf-8') as f:
            json.dump(existing_results, f, ensure_ascii=False, indent=4)
          logger.info(f"  ✓ 저장 완료 (총 {len(existing_results)}개 결과 누적)")
        else:
          # 파싱 실패
          not_parsed_responses.append((p, response))
          logger.warning(f"  ✗ 파싱 실패 - not_parsed.json에 저장")

          # 기존 파일이 있으면 읽어서 누적
          not_parsed_file = f'{output_dir}/{i}/not_parsed.json'
          os.makedirs(os.path.dirname(not_parsed_file), exist_ok=True)
          
          existing_not_parsed = []
          if os.path.exists(not_parsed_file):
            try:
              with open(not_parsed_file, 'r', encoding='utf-8') as f:
                existing_not_parsed = json.load(f)
            except:
              existing_not_parsed = []
          
          # 새 결과 추가
          existing_not_parsed.append((p, response))
          
          # 누적된 결과 저장
          with open(not_parsed_file, 'w', encoding='utf-8') as f:
            json.dump(existing_not_parsed, f, ensure_ascii=False, indent=4)
          logger.warning(f"  ✓ 저장 완료 (총 {len(existing_not_parsed)}개 파싱 실패 누적)")

      except Exception as e:
        # 파싱 실패
        not_parsed_responses.append((p, response))
        logger.error(f"  ✗ 파싱 중 예외 발생: {str(e)}")

        # 기존 파일이 있으면 읽어서 누적
        not_parsed_file = f'{output_dir}/{i}/not_parsed.json'
        os.makedirs(os.path.dirname(not_parsed_file), exist_ok=True)
        
        existing_not_parsed = []
        if os.path.exists(not_parsed_file):
          try:
            with open(not_parsed_file, 'r', encoding='utf-8') as f:
              existing_not_parsed = json.load(f)
          except:
            existing_not_parsed = []
        
        # 새 결과 추가
        existing_not_parsed.append((p, response))
        
        # 누적된 결과 저장
        with open(not_parsed_file, 'w', encoding='utf-8') as f:
          json.dump(existing_not_parsed, f, ensure_ascii=False, indent=4)
        logger.warning(f"  ✓ 저장 완료 (총 {len(existing_not_parsed)}개 파싱 실패 누적)")

    except Exception as e:
      # 응답 없음
      response = None
      no_responses.append(p)
      logger.error(f"  ✗ API 호출 실패: {str(e)}")

      # 기존 파일이 있으면 읽어서 누적
      no_response_file = f'{output_dir}/{i}/no_response.json'
      os.makedirs(os.path.dirname(no_response_file), exist_ok=True)
      
      existing_no_responses = []
      if os.path.exists(no_response_file):
        try:
          with open(no_response_file, 'r', encoding='utf-8') as f:
            existing_no_responses = json.load(f)
        except:
          existing_no_responses = []
      
      # 새 결과 추가
      existing_no_responses.append(p)
      
      # 누적된 결과 저장
      with open(no_response_file, 'w', encoding='utf-8') as f:
        json.dump(existing_no_responses, f, ensure_ascii=False, indent=4)
      logger.warning(f"  ✓ 저장 완료 (총 {len(existing_no_responses)}개 응답 없음 누적)")
    
    logger.info(f"  [{i}개 그룹] {idx}/{len(pick_right_sampling[i])} - 문제 ID: {question_id} 처리 완료")
    # break
  
  logger.info(f"\n[정답 {i}개 그룹] 처리 완료 - 성공: {len(parsed_responses)}, 파싱실패: {len(not_parsed_responses)}, 응답없음: {len(no_responses)}")
  logger.info("=" * 80)
  # break

2025-11-15 14:24:02,770 - INFO - 로그 파일: logs\multiple_answer.log
2025-11-15 14:24:02,770 - INFO - ================================================================================
2025-11-15 14:24:02,772 - INFO - 
[정답 2개 그룹] 처리 시작
2025-11-15 14:24:02,772 - INFO - 
[2개 그룹] 1/666 - 문제 ID: SS0421_q_0691_0002 처리 중...
2025-11-15 14:24:02,773 - INFO -   - API 호출 중... (모델: openai/o3)
2025-11-15 14:24:04,509 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-15 14:25:29,906 - INFO -   - API 응답 수신 완료 (길이: 508자)
2025-11-15 14:25:29,908 - INFO -   - 단일 JSON 객체 파싱 성공
2025-11-15 14:25:29,908 - INFO -   ✓ 파싱 성공 - result.json에 저장
2025-11-15 14:25:29,911 - INFO -   ✓ 저장 완료 (총 1개 결과 누적)
2025-11-15 14:25:29,912 - INFO -   [2개 그룹] 1/666 - 문제 ID: SS0421_q_0691_0002 처리 완료
2025-11-15 14:25:29,912 - INFO - 
[2개 그룹] 2/666 - 문제 ID: SS0405_q_0206_0001 처리 중...
2025-11-15 14:25:29,914 - INFO -   - API 호출 중... (모델: openai/o3)
2025-11-15 14:25:31,035 - INFO - HTTP Req

In [ ]:
# 5문제: 1분 10초